In [5]:
import os
import sys
import pandas as pd
import json
import re
import emoji
import gensim

module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from weat_wefat.src.lib import weat
from weat_wefat.src.lib import weat_tables, weac_helper
import importlib
importlib.reload(weat)
importlib.reload(weat_tables)

<module 'weat_wefat.src.lib.weat_tables' from '/Users/adimaini/Documents/GW/Research/CODE.nosync/weat_wefat/src/lib/weat_tables.py'>

In [4]:
!pip install gensim==4.0

     |████████████████████████████████| 23.1 MB 14.0 MB/s eta 0:00:01
Using legacy 'setup.py install' for gensim, since package 'wheel' is not installed.
  Attempting uninstall: gensim
    Found existing installation: gensim 3.8.3
    Uninstalling gensim-3.8.3:
      Successfully uninstalled gensim-3.8.3
    Running setup.py install for gensim ... done


# C1 corpus testing

In [6]:
ND_PATH = '../../data/us_corpus/c1_tweet_harvesting/ND_CAS_tweets_10-2020_03-2021_c1.jsonl'
nd_data = []
with open(ND_PATH, 'r') as data:
    for line in data: 
        nd_data.append(json.loads(line))

In [8]:
len(nd_data)

34963

In [11]:
text = combined_full_text(nd_data)

In [12]:
text[:100]

'und fans waiting to see what’s left of our team... https://t.co/2hvk6rtjru who do i take in the firs'

In [13]:
nd_data[0]

{'geo': {'place_id': '5c8fea63bc3faa5b',
  'full_name': 'Fargo, ND',
  'geo': {'type': 'Feature',
   'bbox': [-96.903853, 46.787881, -96.756332, 46.941884],
   'properties': {}},
  'country': 'United States',
  'place_type': 'city',
  'name': 'Fargo',
  'country_code': 'US',
  'id': '5c8fea63bc3faa5b'},
 'conversation_id': '1377045670331363331',
 'public_metrics': {'retweet_count': 0,
  'reply_count': 0,
  'like_count': 6,
  'quote_count': 0},
 'entities': {'urls': [{'start': 51,
    'end': 74,
    'url': 'https://t.co/2HvK6rtJrU',
    'expanded_url': 'https://twitter.com/saraj323/status/1377045670331363331/photo/1',
    'display_url': 'pic.twitter.com/2HvK6rtJrU'}]},
 'created_at': '2021-03-30T23:50:46.000Z',
 'author_id': '22858102',
 'reply_settings': 'everyone',
 'attachments': {'media_keys': ['16_1377045664102809601'],
  'media': [{'height': 176,
    'type': 'animated_gif',
    'width': 176,
    'media_key': '16_1377045664102809601',
    'preview_image_url': 'https://pbs.twimg.com

In [38]:
# loading in the binary model export
nd_model = gensim.models.KeyedVectors.load_word2vec_format("../../ND_CAS_model.bin", binary=True)

In [49]:
len(nd_model.vocab)

7326

In [57]:
len(' '.join(ny_filtered[1:5]))

TypeError: sequence item 0: expected str instance, list found

In [47]:
list(nd_model.vocab.keys())

['the',
 'to',
 'a',
 'i',
 'and',
 'in',
 'of',
 'is',
 'for',
 'you',
 'this',
 'my',
 'on',
 'it',
 'with',
 'that',
 'be',
 'at',
 'are',
 'just',
 'have',
 'so',
 'me',
 'we',
 'not',
 'your',
 'all',
 'was',
 '@',
 'but',
 'happy',
 'from',
 'what',
 'out',
 'up',
 'like',
 'will',
 'get',
 'if',
 'they',
 'about',
 'one',
 'as',
 'our',
 'i’m',
 'do',
 '&;',
 'can',
 'by',
 'love',
 'day',
 'time',
 'how',
 'he',
 'good',
 'an',
 'has',
 'north',
 'when',
 'it’s',
 'or',
 'his',
 'new',
 'now',
 'who',
 'dakota',
 'today',
 'no',
 'some',
 'go',
 'people',
 'great',
 'fargo',
 'game',
 'am',
 'more',
 'fargo,',
 'first',
 'see',
 'been',
 '️',
 'know',
 '-',
 'back',
 'going',
 'their',
 'would',
 'posted',
 'don’t',
 'very',
 'had',
 'photo',
 'year',
 'got',
 'still',
 'why',
 'here',
 'need',
 'birthday',
 'make',
 'after',
 'there',
 '2',
 'want',
 'really',
 'think',
 'last',
 'us',
 'these',
 'thank',
 'never',
 'over',
 'best',
 'off',
 'should',
 "i'm",
 'right',
 'her',

# NY-TX corpus

In [60]:
TX_PATH = '../../data/us_corpus/ny_tx_corpus/texas tweets.jsonl'
NY_PATH = '../../data/us_corpus/ny_tx_corpus/NY tweets.jsonl'

In [61]:
# reading in the json files 
tx_data = []
ny_data = []
with open(TX_PATH, 'r') as data:
    for line in data: 
        tx_data.append(json.loads(line))

with open(NY_PATH, 'r') as data:
    for line in data:
        ny_data.append(json.loads(line))

In [62]:
def combined_full_text(json_object):
    corpus = [tweet['full_text'] for tweet in json_object]
    corpus = ' '.join(corpus).lower()
    return corpus

def delete_links(text): 
    return re.sub(r'http\S+', '', text)

def delete_hashtags(text):
    return re.sub(r'#(\w+)', '', text)

def delete_mentions(text):
    return re.sub(r'@(\w+)', '', text)

def delete_emojies(text):
    new_text = re.sub(emoji.get_emoji_regexp(), r"", text)
    return new_text

def delete_new_line_chars(text):
    return " ".join(text.splitlines())

def delete_multiple_spaces(text):
    return " ".join(text.split())

def replace_amp(text):
    return text.replace('&amp', '&')

In [63]:
def split_string_to_list_sentences(filtered):
    '''split strings to a list of sentences'''
    new_text = [text.split() for text in filtered.split('.')]
    # to delete any empty lists in nested list
    new_text = [text for text in new_text if text != []]
    return new_text

In [64]:
def write_filtered_file(filtered_text, state):
    '''write filtered files to disk'''
    with open('../../data/us_corpus/' + state + '_filtered_sentences', 'w') as text_file:
        text_file.write(filtered_text)
    print ('Written successfully.')

In [65]:
tx_filtered = combined_full_text(tx_data)
tx_filtered = delete_links(tx_filtered)
tx_filtered = delete_hashtags(tx_filtered)
tx_filtered = delete_mentions(tx_filtered)
tx_filtered = delete_emojies(tx_filtered)
tx_filtered = delete_new_line_chars(tx_filtered)
tx_filtered = delete_multiple_spaces(tx_filtered)
tx_filtered = replace_amp(tx_filtered)
tx_filtered = split_string_to_list_sentences(tx_filtered)

In [66]:
ny_filtered = combined_full_text(ny_data)
ny_filtered = delete_links(ny_filtered)
ny_filtered = delete_hashtags(ny_filtered)
ny_filtered = delete_mentions(ny_filtered)
ny_filtered = delete_emojies(ny_filtered)
ny_filtered = delete_new_line_chars(ny_filtered)
ny_filtered = delete_multiple_spaces(ny_filtered)
ny_filtered = replace_amp(ny_filtered)
ny_filtered = split_string_to_list_sentences(ny_filtered)

### 814k words in TX dataset and 835k words in NY dataset

We need filters to get rid of: <br>
1. links - done 
2. emojies - done 
3. new line characters and other characters - done 
4. multiple spaces - done

Create word embeddings. 

Processing steps: 
1. Lowercase all words
2. Filtering out tweets
3. skipgram to create one word out of common 2 or 3 word sets. 

Need to make sure the vocabulary is the same between the words. There are 9993 words in tx model, and 10661 words in ny model. There are 7822 words that these models share. I expect in larger embeddings, this % would even go up. 

In [67]:
def create_model(filtered_sentences):
    return gensim.models.Word2Vec(filtered_sentences, size=300)

In [68]:
def get_common_words(model1, model2):
    '''returns the common vocab words between two Word2Vec embeddings'''
    return [word for word in model1.wv.vocab.keys() if word in model2.wv.vocab.keys()]

In [69]:
ny_model = create_model(ny_filtered)
tx_model = create_model(tx_filtered)

In [70]:
# these are words that are available in both tx_model and ny_model
common_words = get_common_words(ny_model, tx_model)

In [ ]:
len(common_words)

In [193]:
keywords = ['republicans', 'democrats', 'covid', 'stay', 'home', 'media', 'quarantine', 'pandemic', 'gatherings', 'mask', 'face', 'government', 'conspiracy', 'vaccine', 'fascist', 'china', 'asian', 'biden']


In [195]:
importlib.reload(weat)
importlib.reload(weat_tables)

<module 'weat_wefat.src.lib.weat_tables' from '/Users/adimaini/Documents/GW/Research/CODE.nosync/weat_wefat/src/lib/weat_tables.py'>

In [196]:
filtered_common_words = [word for word in common_words if word not in keywords]

In [197]:
# weac scores
weac_loading.get_weac_scores(tx_model, ny_model, keywords, filtered_common_words)

{'republicans': (-0.42262019664862915, 0.0),
 'democrats': (0.3788063147772656, 0.0),
 'covid': (-0.3065858810925095, 0.0),
 'stay': (-0.05948491860899714, 0.0),
 'home': (-0.17204698073223523, 0.0),
 'media': (-0.1423816553849855, 0.0),
 'quarantine': (-0.44688170068873834, 1.0),
 'pandemic': (-0.08786795995007747, 1.0),
 'gatherings': (-0.0891795678418143, 1.0),
 'mask': (0.7496289856443449, 0.0),
 'face': (1.0349580878465006, 0.0),
 'government': (-0.114478569130066, 0.8734845164093847),
 'conspiracy': (-0.25916876172570374, 0.23094297137501274),
 'vaccine': (0.3538996158602639, 0.0),
 'fascist': (-0.2842446625603819, 0.028645589186632736),
 'china': (-0.2815641394100684, 1.0),
 'asian': (0.18241944125687906, 1.0),
 'biden': (1.0187941299612506, 0.0)}

Reading in the weac .json objects 

In [199]:
with open('../../data/us_corpus/weac_ny_tx_covid_tweets.json_4.json', 'r') as f:
    ny_tx_weac_scores = json.load(f)
ny_tx_weac_df = pd.DataFrame.from_dict(ny_tx_weac_scores, orient='index', columns=['effect size', 'p-value'])
    


## weac results

In [200]:
ny_tx_weac_df.sort_values('effect size')

,effect size,p-value
quarantine,-0.446657,1.000000
covid,-0.306351,0.000000
fascist,-0.284131,0.000205
china,-0.281601,1.000000
conspiracy,-0.258967,0.072358
home,-0.172125,0.000000
media,-0.142106,0.000000
government,-0.114451,0.865592
gatherings,-0.089238,1.000000
pandemic,-0.087914,1.000000


In [67]:
tx_model.wv.save_word2vec_format('../../data/us_corpus/tx_model.bin')
ny_model.wv.save_word2vec_format('../../data/us_corpus/ny_model.bin')

Running WEAT tests on these two sets of embeddings

In [71]:
df_tx = weat_tables.output_table(model=tx_model,
                                filepath = '../../targets_attributes_data/covid related/covid pleasant unpleasant.csv',
                                algorithm = 'wefat')

Reading files...

Finished.


In [72]:
df_ny = weat_tables.output_table(model=ny_model,
                                filepath = '../../targets_attributes_data/covid related/covid pleasant unpleasant.csv',
                                algorithm = 'wefat')

Reading files...

Finished.


In [73]:
df_tx

,Target,Effect Size,P-Value
0,gatherings,-0.33,0.756395
1,mask,1.07,0.011841
2,distancing,0.42,0.198064
3,restrictions,-0.45,0.813507
4,quarantine,0.25,0.314287
5,parties,-0.48,0.829870
6,covid,-0.76,0.939065
7,vaccine,-1.11,0.988328
8,indoors,0.42,0.200892


In [74]:
df_ny

,Target,Effect Size,P-Value
0,gatherings,-0.26,0.699907
1,mask,0.33,0.264429
2,distancing,-0.11,0.582624
3,restrictions,-0.48,0.825837
4,quarantine,0.21,0.336840
5,parties,-0.42,0.809072
6,covid,-0.86,0.960895
7,vaccine,-1.20,0.990920
8,indoors,-0.65,0.909384


In [77]:
merged = pd.merge(df_tx, df_ny, on='Target', suffixes=('_tx', '_ny'))

In [79]:
merged['effect_size_difference'] = merged['Effect Size_tx'] - merged['Effect Size_ny']

In [82]:
merged.sort_values(by='effect_size_difference', ascending=False)

,Target,Effect Size_tx,P-Value_tx,Effect Size_ny,P-Value_ny,effect_size_difference
8,indoors,0.42,0.200892,-0.65,0.909384,1.07
1,mask,1.07,0.011841,0.33,0.264429,0.74
2,distancing,0.42,0.198064,-0.11,0.582624,0.53
6,covid,-0.76,0.939065,-0.86,0.960895,0.10
7,vaccine,-1.11,0.988328,-1.20,0.990920,0.09
4,quarantine,0.25,0.314287,0.21,0.336840,0.04
3,restrictions,-0.45,0.813507,-0.48,0.825837,0.03
5,parties,-0.48,0.829870,-0.42,0.809072,-0.06
0,gatherings,-0.33,0.756395,-0.26,0.699907,-0.07
